# Structured Outputs for LLMs

This guide demonstrates how to use structured outputs with LLMs and provides a practical implementation to further understand the topic.

## What are Structured Outputs?

Structured outputs enable LLMs to generate responses that adhere to a specific JSON schema (this can be provided using Pydantic). This ensures that:

1. **Responses are consistently formatted** - The model returns data in a predictable structure
2. **Type-safety is maintained** - No need to validate or retry incorrectly formatted responses
3. **Integration with applications is simplified** - Data can be directly fed into downstream processes

In [1]:
# Import warnings
import warnings
warnings.filterwarnings("ignore")

from utils import *
from pydantic import BaseModel

## Simple Example

In [2]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [3]:
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


## The Use Cases

- Tasks that require reasoning (step-by-step outputs)
- Unstructured data --> structured data
- UI Generation (e.g., generate valid HTML)
- Content moderation

## CoT Example

In [4]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

system_message = "You are a helpful math tutor. Guide the user through the solution step by step."
query = "how can I solve 8x + 7 = -23"

get_structured_output(query, system_message, MathReasoning, "math reasoning")   

MathReasoning(steps=[Step(explanation='Start with the equation: \\(8x + 7 = -23\\)', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to isolate the term with \\(x\\) on the left side. This gives you: \\(8x + 7 - 7 = -23 - 7\\).', output='8x = -30'), Step(explanation='Now, divide both sides of the equation by 8 to solve for \\(x\\).', output='x = \\frac{-30}{8}'), Step(explanation='Simplify \\(\\frac{-30}{8}\\) by dividing the numerator and the denominator by their greatest common divisor, which is 2.', output='x = \\frac{-15}{4}')], final_answer='x = -\\frac{15}{4}')

## Tips for Data Structure

To maximize the quality of model generations, OpenAI recommends the following:

- Name keys clearly and intuitively
- Create clear titles and descriptions for important keys in your structure
- Create and use evals to determine the structure that works best for your use case

## Content Moderation Example

In [5]:
from enum import Enum
from typing import Optional

class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

system_message = "Determine if the user input violates specific guidelines and explain if they do."
query = "How do I prepare for a job interview?"

get_structured_output(query, system_message, ContentCompliance, "content compliance")

ContentCompliance(is_violating=False, category=None, explanation_if_violating=None)

## Handling Refusals

As with other calls, the structured output with user inputs could lead to refusals, for instance, because of safety reasons. To handle this in your application, you can use a conditional logic to check if the `refusal` property is present in the structured output.

If there is a refusal, the output will look different.

In [6]:
# users the same schemas defined in the previous CoT example

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if (math_reasoning.refusal):
    print(math_reasoning.refusal)
else:
    print(math_reasoning.parsed)

steps=[Step(explanation="The goal is to solve for \\( x \\) in the equation \\( 8x + 7 = -23 \\). Start by isolating the term with \\( x \\). Since the constant term on the left side is \\( +7 \\), we'll subtract 7 from both sides of the equation to remove it.", output='Perform: \\( 8x + 7 - 7 = -23 - 7 \\)'), Step(explanation='Subtracting 7 from both sides simplifies to \\( 8x = -30 \\). Now, we have an equation with the variable term \\( 8x \\) by itself on one side.', output='Result: \\( 8x = -30 \\)'), Step(explanation='To solve for \\( x \\), divide both sides by 8, the coefficient of \\( x \\). This will isolate \\( x \\).', output='Perform: \\( \\frac{8x}{8} = \\frac{-30}{8} \\)'), Step(explanation='Simplifying \\( \\frac{8x}{8} \\) gives \\( x \\), and simplifying \\( \\frac{-30}{8} \\) results in \\( -\\frac{15}{4} \\), which is the value of \\( x \\).', output='Result: \\( x = -\\frac{15}{4} \\)')] final_answer='x = -\\frac{15}{4}'


## Handling Incompatible Inputs & Mistakes

For open-ended applications that accept user-generated inputs, make sure to deal with incompatible requests. 

In [7]:
system_message = "You are a helpful math tutor. Guide the user through the solution step by step."
query = "What's the weather today?"

get_structured_output(query, system_message, MathReasoning, "math reasoning")   

MathReasoning(steps=[Step(explanation="The user's question asks about the weather today, which isn't related to solving a math problem. I don't have the capability to provide weather updates or forecasts.", output="The user's question is outside the scope of math problem-solving.")], final_answer="I'm unable to provide weather updates. Please check a reliable weather source.")

The model will always try to adhere to the schema provided. It can results in hallucinations so to mitigate this issue you can improve the system prompt:

In [8]:
system_message = "You are a helpful math tutor. Guide the user through the solution step by step. Return empty parameters if the request is invalid."
query = "What's the weather today?"

get_structured_output(query, system_message, MathReasoning, "math reasoning")   

MathReasoning(steps=[], final_answer='')

The structured outputs feature is not perfect and might require additional tuning of the system prompt/instructions. Breaking the task into simpler subtasks, as we do in CoT, can also help. 

Check out the documentation for further tips and best practices: https://platform.openai.com/docs/guides/structured-outputs#supported-schemas